# Chapter 2 - Working with text data

# Downloading data

```python
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "../datasets/the-verdict.txt"
urllib.request.urlretrieve(url, file_path)
```

In [1]:
import re

In [2]:
with open(file="../datasets/the-verdict.txt", mode="r", encoding="utf-8") as f:
    raw_text = f.read()

In [3]:
print(f"Total number of character: {len(raw_text)}")
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [4]:
text = "Hello, world. This, is a test."
result = re.split(r"(\s)", text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [ ]:
result = re.split(r"([,.]|\s)", text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [ ]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [8]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [9]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4690


In [10]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [11]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [12]:
vocab = {token: idx for idx, token in enumerate(all_words)}

In [18]:
from __future__ import annotations

from typing import List


class SimpleTokenizerV1:

    def __init__(self, vocab: dict) -> SimpleTokenizerV1:

        self.vocab = vocab
        self.idx_to_token = {idx: token for token, idx in vocab.items()}

    def encode(self, text: List[str]) -> str:

        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.vocab[s] for s in preprocessed]

        return ids
    
    def decode(self, ids: List[int]) -> str:

        text = " ".join([self.idx_to_token[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)

        return text

In [19]:
tokenizer = SimpleTokenizerV1(vocab=vocab)

In [20]:
text = """"It's the last he painted, you know,"
Mrs. Gisburn said with pardonable pride."""

In [21]:
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [22]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [23]:
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

KeyError: 'Hello'